In [ ]:
import requests
from IPython.core.display import HTML
from bs4 import BeautifulSoup
from bs4.element import Tag

In [ ]:
#פָּעַ for pa'al, פִּעֵ pi'el, הִפְ hif'il, הִתְ hitpa'el, הֻפְ huf'al, פֻּעַ pu'al, נִפְ nifal

In [ ]:
url = 'https://www.pealim.com/dict/1843-letzaref/'

In [ ]:
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')

In [ ]:
table = soup.select('.conjugation-table.table')[0]
HTML(str(table))

In [ ]:
table_body = table.find('tbody')

content = {}

def parse_row(row):
    return [parse_cell(cell) for cell in row.find_all('td')]

def parse_cell(cell):
    
    for div in cell.find_all("div", {'class':'aux-forms'}): 
        div.decompose()

    result = {'hebrew' : [str(h.contents[0]) for h in cell.select('.menukad')],
              'transcript' : [''.join([str(s) for s in h.contents]) for h in cell.select('.transcription')]}
    if 'colspan' in cell.attrs:
        result['colspan'] = cell.attrs['colspan']
    return result

In [ ]:
table_content = [parse_row(row) for row in table_body.find_all('tr')]

In [ ]:
table_content

In [ ]:
def create_header_row(name):
    row = Tag(name='tr', attrs={'class' : 'butt'})
    cell = Tag(name='td', attrs={'colspan': '4'})
    cell.append(name)
    row.append(cell)
    return row

def create_cell(cell_content, add_rowspan=False):
    cell_attrs = {}
    if 'colspan' in cell_content:
        cell_attrs['colspan'] = cell_content['colspan']
    if add_rowspan:
        cell_attrs['rowspan'] = '2'
    cell = Tag(name='td', attrs=cell_attrs)
    word_count = len(cell_content['hebrew'])
    
    for i in range(word_count):
        content = Tag(name='ruby')
        content.append(cell_content['hebrew'][i])
        transcript = Tag(name='rt')
        transcript.append(BeautifulSoup(cell_content['transcript'][i], 'html.parser'))
        content.append(transcript)
        cell.append(content)
        if i < word_count - 1:
            cell.append(BeautifulSoup('<br />', 'html.parser'))
    return cell

def create_conjugation_row(row_content, add_rowspan=None):
    row = Tag(name='tr', attrs={'class' : 'cont'})
    if add_rowspan is None:
        add_rowspan = [False] * len(row_content)
    for cell, rs in zip(row_content, add_rowspan):
        row.append(create_cell(cell, rs))
    return row

def create_border_row():
    return Tag(name='tr', attrs={'class' : 'cont bord'})


tbody = Tag(name='tbody')
tbody.append(create_header_row('הוֹוֶה'))
tbody.append(create_conjugation_row(table_content[0]))
tbody.append(create_border_row())
tbody.append(create_header_row('עָבָר'))
tbody.append(create_conjugation_row(table_content[1]))
tbody.append(create_conjugation_row(table_content[2]))
tbody.append(create_conjugation_row(table_content[3]))
tbody.append(create_border_row())
tbody.append(create_header_row('עָתִיד'))
tbody.append(create_conjugation_row(table_content[4]))
tbody.append(create_conjugation_row(table_content[5], add_rowspan=[False, False, False, True]))
tbody.append(create_conjugation_row(table_content[6][:-1]))
if len(table_content) > 7:
    tbody.append(create_border_row())
    tbody.append(create_header_row('צִוּוּי'))
    tbody.append(create_conjugation_row(table_content[7]))

new_table = Tag(name='table')
new_table.append(tbody)

In [ ]:
new_table

In [ ]:
HTML(str(new_table))